In [32]:
import os
import pandas as pd
import numpy as np

In [33]:
df = pd.DataFrame()
for root, _, files in os.walk("data"):
    for filename in files:
        temp_df = pd.read_excel(os.path.join(root, filename), "Sheet1")
        temp_df.drop(columns=[temp_df.columns[0], "Разница в долларах", "Дельта в процентах"], axis=1, inplace=True, errors="ignore")
        df = pd.concat([df, temp_df], axis=0, sort=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24484 entries, 0 to 1497
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Время          16445 non-null  object 
 1   Цена до        16445 non-null  float64
 2   Цена после     16445 non-null  float64
 3   Текст новости  16445 non-null  object 
 4   Аннотация      4780 non-null   object 
 5   Дата           11488 non-null  object 
 6   Unnamed: 1     842 non-null    object 
 7   Unnamed: 2     842 non-null    float64
 8   Unnamed: 3     842 non-null    float64
 9   Unnamed: 4     842 non-null    object 
 10  Unnamed: 5     834 non-null    object 
dtypes: float64(4), object(7)
memory usage: 2.2+ MB


In [36]:
df = df.drop_duplicates(subset=["Текст новости"], keep=False)
df.rename(columns={"Цена до": "price_before", "Цена после": "price_after", "Дата": "date", "Время": "Time", "Текст новости": "news_text", "Аннотация": "annotation"}, inplace=True)

df["absolute_price_difference"] = df["price_after"] - df["price_before"]
df["news_text"] = df["news_text"] + ". " + df["annotation"].fillna("")
df["label"] = np.where(df["absolute_price_difference"] > 0, 1, 0)
df = df[["news_text", "label"]].copy()
df.head()

,news_text,label
0,Spirit AeroSystems Slides On Q1 Earnings Miss ...,1
1,Boeing Unusual Options Activity For May 01. So...,0
2,If You Invested $1000 In This Stock 20 Years A...,0
4,Crude Oil Down 3%; Boeing Reports Better-Than-...,0
5,"US Stocks Rise, Aided By Tech Earnings, But Th...",0


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6112 entries, 0 to 1009
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   news_text  6112 non-null   object
 1   label      6112 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 143.2+ KB


In [38]:
data = pd.read_excel("data/amazon.xlsx", "Sheet1", index_col=None)
data.to_csv("data/amazon.csv", encoding="utf-8", index=False)